In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [56]:
import numpy as np
import pandas as pd
import json
from collections import defaultdict
import os
import io
import distutils.dir_util

from collections import Counter

In [47]:
def write_json(data, fname):
    def _conv(o):
        if isinstance(o, (np.int64, np.int32)):
            return int(o)
        raise TypeError

    parent = os.path.dirname(fname)
    distutils.dir_util.mkpath(parent)

    with io.open(fname, "w", encoding="utf-8") as f:
        json_str = json.dumps(data, ensure_ascii=False, default=_conv)
        f.write(json_str)


def load_json(fname):
    with open(fname, encoding='utf-8') as f:
        json_obj = json.load(f)

    return json_obj

In [ ]:
PATH = f'/content/drive/MyDrive/Test/'
genre_gn_all_dict = load_json(PATH + 'genre_gn_all.json')
song_meta_json = load_json(PATH + 'song_meta.json')

In [55]:
train = load_json(PATH + 'test.json')
val = load_json(PATH + 'val.json')
test = load_json(PATH + 'test.json')

data = train + val + test
data_df = pd.DataFrame(data)
data_df.head()

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[],70107,,"[398985, 449403, 411543, 528044, 143048, 98020...",6,2012-09-29 01:57:26.000
1,"[나만의Best3, 인디아티스트들의추천음악]",7461,,"[196298, 269984, 267805, 175867, 529244, 63825...",0,2019-12-17 14:06:45.000
2,[드라이브],90348,,"[273433, 331003, 68432, 411659, 117793, 616860...",21,2015-05-23 10:44:48.000
3,[분위기],58617,,"[702227, 48152, 440008, 358488, 701041, 540721...",0,2019-03-14 09:47:34.000
4,[],102395,,"[630683, 481582, 528550, 285114, 506667, 17922...",38,2018-07-11 16:43:32.000


In [ ]:
song_df = pd.DataFrame(song_meta_json)

In [ ]:
song_df.head()

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2
3,"[GN1102, GN1101]",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[GN1100],[Matoma],3
4,"[GN1802, GN1801]",20110824,그남자 그여자,2008470,[560160],그남자 그여자,[GN1800],[Jude Law],4


In [ ]:
large_genres = [key for key in genre_gn_all_dict.keys() if key[-2:] == '00']
large_genres

['GN0100',
 'GN0200',
 'GN0300',
 'GN0400',
 'GN0500',
 'GN0600',
 'GN0700',
 'GN0800',
 'GN0900',
 'GN1000',
 'GN1100',
 'GN1200',
 'GN1300',
 'GN1400',
 'GN1500',
 'GN1600',
 'GN1700',
 'GN1800',
 'GN1900',
 'GN2000',
 'GN2100',
 'GN2200',
 'GN2300',
 'GN2400',
 'GN2500',
 'GN2600',
 'GN2700',
 'GN2800',
 'GN2900',
 'GN3000']

In [ ]:
song_df['song_gn_dtl_gnr_basket'] = song_df['song_gn_dtl_gnr_basket'].map(lambda x: tuple(x))
song_df['song_gn_gnr_basket'] = song_df['song_gn_gnr_basket'].map(lambda x: tuple(x))
song_df['artist_id_basket'] = song_df['artist_id_basket'].map(lambda x: tuple(x))
song_df['artist_name_basket'] = song_df['artist_name_basket'].map(lambda x: tuple(x))
song_df['large_genre'] = song_df['song_gn_gnr_basket'].map(lambda x: str(x[0]) if x else x)
song_df.head()

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,large_genre
0,"(GN0901,)",20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,"(2727,)",Feelings,"(GN0900,)","(Various Artists,)",0,GN0900
1,"(GN1601, GN1606)",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,"(29966,)","Bach : Partita No. 4 In D Major, BWV 828 - II....","(GN1600,)","(Murray Perahia,)",1,GN1600
2,"(GN0901,)",20180518,Hit,4698747,"(3361,)",Solsbury Hill (Remastered 2002),"(GN0900,)","(Peter Gabriel,)",2,GN0900
3,"(GN1102, GN1101)",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,"(838543,)",Feeling Right (Everything Is Nice) (Feat. Popc...,"(GN1100,)","(Matoma,)",3,GN1100
4,"(GN1802, GN1801)",20110824,그남자 그여자,2008470,"(560160,)",그남자 그여자,"(GN1800,)","(Jude Law,)",4,GN1800


In [ ]:
song_df.head()

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,large_genre
0,"(GN0901,)",20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,"(2727,)",Feelings,"(GN0900,)","(Various Artists,)",0,GN0900
1,"(GN1601, GN1606)",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,"(29966,)","Bach : Partita No. 4 In D Major, BWV 828 - II....","(GN1600,)","(Murray Perahia,)",1,GN1600
2,"(GN0901,)",20180518,Hit,4698747,"(3361,)",Solsbury Hill (Remastered 2002),"(GN0900,)","(Peter Gabriel,)",2,GN0900
3,"(GN1102, GN1101)",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,"(838543,)",Feeling Right (Everything Is Nice) (Feat. Popc...,"(GN1100,)","(Matoma,)",3,GN1100
4,"(GN1802, GN1801)",20110824,그남자 그여자,2008470,"(560160,)",그남자 그여자,"(GN1800,)","(Jude Law,)",4,GN1800


In [ ]:
genre_gn_all_dict

{'GN0100': '발라드',
 'GN0101': '세부장르전체',
 'GN0102': "'80",
 'GN0103': "'90",
 'GN0104': "'00",
 'GN0105': "'10-",
 'GN0200': '댄스',
 'GN0201': '세부장르전체',
 'GN0202': "'80",
 'GN0203': "'90",
 'GN0204': "'00",
 'GN0205': "'10-",
 'GN0300': '랩/힙합',
 'GN0301': '세부장르전체',
 'GN0302': '랩 스타일',
 'GN0303': '보컬 스타일',
 'GN0304': '언더그라운드 힙합',
 'GN0305': '시대별',
 'GN0400': 'R&B/Soul',
 'GN0401': '세부장르전체',
 'GN0402': '어반',
 'GN0403': 'R&B',
 'GN0500': '인디음악',
 'GN0501': '세부장르전체',
 'GN0502': '포크',
 'GN0503': '록',
 'GN0504': '일렉',
 'GN0505': '힙합',
 'GN0506': '발라드',
 'GN0507': "'90",
 'GN0508': "'00",
 'GN0509': "'10-",
 'GN0600': '록/메탈',
 'GN0601': '세부장르전체',
 'GN0602': "'70",
 'GN0603': "'80",
 'GN0604': "'90",
 'GN0605': "'00",
 'GN0606': "'10-",
 'GN0700': '성인가요',
 'GN0701': '세부장르전체',
 'GN0702': '신세대트로트',
 'GN0703': '전설의트로트',
 'GN0704': '뽕짝트로트',
 'GN0705': '트로트메들리',
 'GN0706': "트로트'60-'70",
 'GN0707': "트로트'80-'90",
 'GN0708': "트로트'00-",
 'GN0709': "성인가요'80-'90",
 'GN0710': "성인가요'00-",
 'GN0800': '포크/블루스',

In [ ]:
song_gb = song_df.groupby('large_genre')
song_PATH = PATH + 'Song_by_Genres/'
if not os.path.exists(song_PATH):
    os.mkdir(song_PATH)

for key, group in song_gb:
    if not key:
        key = 'GN0000_장르없음'
    elif key == 'GN9000':
        key += '_장르모름'
    else:
        _fix = genre_gn_all_dict[key]
        _fix = _fix.replace('/',  ' ')
        key += '_' + _fix
    group_json = group.to_json(song_PATH + str(key) + '.json', force_ascii=False)

In [58]:
data_df['songs_counter'] = data_df['songs'].map(lambda x: Counter(x))

data_df.head()

,tags,id,plylst_title,songs,like_cnt,updt_date,songs_counter
0,[],70107,,"[398985, 449403, 411543, 528044, 143048, 98020...",6,2012-09-29 01:57:26.000,"{398985: 1, 449403: 1, 411543: 1, 528044: 1, 1..."
1,"[나만의Best3, 인디아티스트들의추천음악]",7461,,"[196298, 269984, 267805, 175867, 529244, 63825...",0,2019-12-17 14:06:45.000,"{196298: 1, 269984: 1, 267805: 1, 175867: 1, 5..."
2,[드라이브],90348,,"[273433, 331003, 68432, 411659, 117793, 616860...",21,2015-05-23 10:44:48.000,"{273433: 1, 331003: 1, 68432: 1, 411659: 1, 11..."
3,[분위기],58617,,"[702227, 48152, 440008, 358488, 701041, 540721...",0,2019-03-14 09:47:34.000,"{702227: 1, 48152: 1, 440008: 1, 358488: 1, 70..."
4,[],102395,,"[630683, 481582, 528550, 285114, 506667, 17922...",38,2018-07-11 16:43:32.000,"{630683: 1, 481582: 1, 528550: 1, 285114: 1, 5..."


In [76]:
songs_counter = Counter()
for playlist in data:
    songs_counter += Counter(playlist['songs'])

In [79]:
songs_mp_counter = songs_counter.most_common()
songs_mp_counter

[(116573, 352),
 (144663, 317),
 (366786, 309),
 (357367, 296),
 (349492, 266),
 (650494, 266),
 (549178, 264),
 (701557, 262),
 (133143, 259),
 (627363, 257),
 (523521, 252),
 (663256, 251),
 (42155, 242),
 (463173, 242),
 (13281, 241),
 (300087, 239),
 (654757, 239),
 (610933, 238),
 (520093, 237),
 (396828, 237),
 (209135, 235),
 (680366, 234),
 (427724, 234),
 (152422, 234),
 (105140, 234),
 (448116, 232),
 (199262, 231),
 (675115, 230),
 (625875, 230),
 (68348, 229),
 (187047, 228),
 (461341, 228),
 (422915, 227),
 (547967, 226),
 (449244, 226),
 (339124, 226),
 (582252, 226),
 (75842, 225),
 (11657, 224),
 (132994, 224),
 (215411, 222),
 (418935, 221),
 (485155, 219),
 (13198, 219),
 (519391, 217),
 (509998, 216),
 (422077, 215),
 (169984, 215),
 (678762, 214),
 (668128, 213),
 (505036, 213),
 (341513, 213),
 (348200, 212),
 (442014, 212),
 (350309, 211),
 (487911, 211),
 (37748, 210),
 (643628, 210),
 (140867, 210),
 (581799, 210),
 (224921, 209),
 (157055, 208),
 (209993, 206),

In [96]:
song_sub_df = song_df[['id', 'large_genre']]
song_sub_df.iloc[900].large_genre

'GN1900'

In [102]:
songs_mc_counter = songs_counter.most_common()


top_songs = {genre: [] for genre in large_genres}
top_songs["GN0000"] = []
top_songs["GN9000"] = []
for song_id, cnt in songs_mc_counter:
    lg = song_sub_df.iloc[song_id].large_genre
    try:
        top_songs[lg].append((song_id, cnt))
    except KeyError:
        if lg == ():
            lg = 'GN0000'
            top_songs[lg].append((song_id, cnt))

In [130]:
top_20_songs = {genre: songs[:20] for genre, songs in top_songs.items()}

top_20_songs

{'GN0000': [(523597, 17),
  (293156, 15),
  (143168, 13),
  (575515, 13),
  (85336, 13),
  (291377, 12),
  (293582, 11),
  (243994, 9),
  (457755, 9),
  (478287, 8),
  (650937, 8),
  (311105, 8),
  (289796, 7),
  (202341, 6),
  (375111, 6),
  (521276, 6),
  (309585, 6),
  (350840, 5),
  (44137, 5),
  (489662, 4)],
 'GN0100': [(144663, 317),
  (349492, 266),
  (650494, 266),
  (549178, 264),
  (627363, 257),
  (42155, 242),
  (463173, 242),
  (520093, 237),
  (396828, 237),
  (680366, 234),
  (427724, 234),
  (105140, 234),
  (448116, 232),
  (199262, 231),
  (675115, 230),
  (68348, 229),
  (187047, 228),
  (422915, 227),
  (582252, 226),
  (75842, 225)],
 'GN0200': [(133484, 158),
  (326204, 130),
  (455668, 128),
  (345004, 124),
  (122363, 122),
  (138031, 106),
  (89620, 94),
  (26797, 90),
  (61595, 86),
  (354659, 82),
  (165434, 79),
  (461680, 77),
  (589292, 75),
  (506254, 74),
  (548896, 74),
  (431902, 73),
  (682550, 72),
  (430005, 71),
  (664555, 71),
  (420005, 71)],
 '

In [145]:
song_df['count'] = 0
for key in top_20_songs.keys():
    tmp_PATH = PATH + 'Top_20_Songs/'
    if not os.path.exists(tmp_PATH):
        os.mkdir(tmp_PATH)
    
    tmp_df = pd.DataFrame(columns=song_df.columns)
    for id, cnt in top_20_songs[key]:
        song = song_df[song_df['id'] == id]
        song['count'] = cnt
        tmp_df = tmp_df.append(song)
    
    try:
        _fix = genre_gn_all_dict[key]
        _fix = _fix.replace('/',  ' ')
    except KeyError:
        if key == 'GN0000':
            _fix = '장르없음'
        elif key == 'GN9000':
            _fix = '장르모름'
    
    tmp_df.to_json(tmp_PATH + str(key) + '_' + _fix + '.json', force_ascii=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

In [116]:
genre_gn_all_dict

{'GN0100': '발라드',
 'GN0101': '세부장르전체',
 'GN0102': "'80",
 'GN0103': "'90",
 'GN0104': "'00",
 'GN0105': "'10-",
 'GN0200': '댄스',
 'GN0201': '세부장르전체',
 'GN0202': "'80",
 'GN0203': "'90",
 'GN0204': "'00",
 'GN0205': "'10-",
 'GN0300': '랩/힙합',
 'GN0301': '세부장르전체',
 'GN0302': '랩 스타일',
 'GN0303': '보컬 스타일',
 'GN0304': '언더그라운드 힙합',
 'GN0305': '시대별',
 'GN0400': 'R&B/Soul',
 'GN0401': '세부장르전체',
 'GN0402': '어반',
 'GN0403': 'R&B',
 'GN0500': '인디음악',
 'GN0501': '세부장르전체',
 'GN0502': '포크',
 'GN0503': '록',
 'GN0504': '일렉',
 'GN0505': '힙합',
 'GN0506': '발라드',
 'GN0507': "'90",
 'GN0508': "'00",
 'GN0509': "'10-",
 'GN0600': '록/메탈',
 'GN0601': '세부장르전체',
 'GN0602': "'70",
 'GN0603': "'80",
 'GN0604': "'90",
 'GN0605': "'00",
 'GN0606': "'10-",
 'GN0700': '성인가요',
 'GN0701': '세부장르전체',
 'GN0702': '신세대트로트',
 'GN0703': '전설의트로트',
 'GN0704': '뽕짝트로트',
 'GN0705': '트로트메들리',
 'GN0706': "트로트'60-'70",
 'GN0707': "트로트'80-'90",
 'GN0708': "트로트'00-",
 'GN0709': "성인가요'80-'90",
 'GN0710': "성인가요'00-",
 'GN0800': '포크/블루스',